In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import kfac
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from plotting import plot_3d, plot_2d
%matplotlib widget
tf.__version__


def load_numpy_as_iterator(tensor, num_epochs, batch_size, dtype=tf.float32):
    x = tf.data.Dataset.from_tensor_slices(tf.convert_to_tensor(tensor, dtype=dtype)).repeat(num_epochs).batch(batch_size)
    x = tf.compat.v1.data.make_one_shot_iterator(x).get_next()
    return x



/home/maxiwelian/anaconda3/envs/tf1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/maxiwelian/anaconda3/envs/tf1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/maxiwelian/anaconda3/envs/tf1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/maxiwelian/anaconda3/envs/tf1/lib/pyt

In [2]:
input_dim = 2
output_dim = 2

input_data = np.load('data/input.npy').astype(np.float32)
labels = np.load('data/labels.npy').astype(np.int32)
labels = np.reshape(labels, (-1,))

weights = np.load('data/weights.npy').astype(np.float32)

x = tf.placeholder("float", [None, input_dim])
y = tf.placeholder("int32", [None])

# input_data = tf.convert_to_tensor(input_data)
# labels = tf.convert_to_tensor(labels)



In [3]:
# input_dim = 2
# output_dim = 2

# input_data = np.load('data/input.npy').astype(np.float32)
# labels = np.load('data/labels.npy').astype(np.int32)
# labels = np.reshape(labels, (-1,))

# weights = np.load('data/weights.npy').astype(np.float32)

# x = tf.placeholder("float", [None, input_dim])
# y = tf.placeholder("int32", [None])
# w = tf.placeholder("float", [input_dim, output_dim])

# layer = tf.layers.Dense(output_dim, weights=[weights], name="fc", use_bias=False, activation=None)
# logits = layer(x)
# loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits))

# config = tf.ConfigProto(allow_soft_placement=True)
# with tf.train.MonitoredTrainingSession(config=config) as sess:
#     while not sess.should_stop():
#         loss_ = sess.run(loss, feed_dict={x:input_data, y:labels})
# print('Loss', loss_)

In [6]:
input_dim = 2
output_dim = 2

input_data = np.load('data/input.npy').astype(np.float32)
labels = np.load('data/labels.npy').astype(np.int32)
labels = np.reshape(labels, (-1,))

weights = np.load('data/weights.npy').astype(np.float32)

input_data = tf.convert_to_tensor(input_data)
labels = tf.convert_to_tensor(labels)

layer = tf.layers.Dense(output_dim, weights=[weights], use_bias=False)
logits = layer(input_data)
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))

layer_collection = kfac.LayerCollection()
layer_collection.register_softmax_cross_entropy_loss(logits, name="logits")
layer_collection.auto_register_layers()

config = tf.ConfigProto(allow_soft_placement=True)
with tf.train.MonitoredTrainingSession(config=config) as sess:
    loss_ = sess.run(loss)
print('Loss', loss_)

AmbiguousRegistrationError: Tried to register <tf.Variable 'dense/kernel:0' shape=(2, 2) dtype=float32_ref> as generic without knowledge of batch_size. You can pass batch_size in to fix this error. But please note, generic registrations may be a symptom that the scanner is failing to auto-detect your model. Generic uses a last-resort approximation, and should never be used for common layer types that K-FAC properly supports, such as convs or fully-connected layers.

In [5]:
layer = tf.layers.Dense(output_dim, weights=[weights], name="fc", use_bias=False)
logits = layer(input_data)
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
layer_collection = kfac.LayerCollection()
layer_collection.register_softmax_cross_entropy_loss(logits, name="logits")
layer_collection.auto_register_layers()

AmbiguousRegistrationError: Tried to register <tf.Variable 'dense/kernel:0' shape=(2, 2) dtype=float32_ref> as generic without knowledge of batch_size. You can pass batch_size in to fix this error. But please note, generic registrations may be a symptom that the scanner is failing to auto-detect your model. Generic uses a last-resort approximation, and should never be used for common layer types that K-FAC properly supports, such as convs or fully-connected layers.

### First lets compare the pure loss computation

### This time with kfac

In [ ]:
kfac_optimizer = kfac.PeriodicInvCovUpdateKfacOpt(learning_rate=0.00, damping=0.001, layer_collection=layer_collection)
kfac_train_op = kfac_optimizer.minimize(loss)

In [ ]:

cov_vars = kfac_optimizer.get_cov_vars()

history = []
losses = []
with tf.train.MonitoredTrainingSession() as sess:
    for i in range(100):
        loss_, converged_, covs_ = sess.run([loss, kfac_train_op, cov_vars])
        losses.append(loss_)

In [ ]:
losses

In [ ]:
plt.close()
plot_3d(history, np.load('data/input.npy'), np.load('data/target.npy'))
plt.show()

In [ ]:
plt.close()
plt.title('TF1 KFAC')
plot_2d(history, np.load('data/input.npy'), np.load('data/target.npy'), (-0.5, 1, -3, 1))
plt.show()